In [ ]:
import re

import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from ploting import *
import json

In [ ]:
root_path = "frontier"
gccl = {"frontier": "RCCL",
        "perlmutter": "NCCL"}[root_path]

In [ ]:
name_re = re.compile("timings\.(\d+)\.json")

In [ ]:
data = json.load(open(f"frontier/data/timings.16.json"))
data["benchmarks"]

In [ ]:
def to_seconds(t, unit):
    map = {'ns': 1e-9}
    return float(t) * map[unit]

In [ ]:
def flatten(json_data, extra_data=None):
    extra_data = extra_data or dict()
    flat_data = []
    for run in json_data["benchmarks"]:
        full_name = run["name"].split("/")
        operation = full_name[0]
        comm = full_name[1]
        msg_size = full_name[2]
        kernels = full_name[3]

        flat_run = {}
        flat_run.update(operation=operation, comm=comm, msg_size=int(msg_size), kernels=int(kernels),
                        real_time=to_seconds(run["real_time"], run["time_unit"]),
                        cpu_time=to_seconds(run["real_time"], run["time_unit"]),
                        **extra_data)
        flat_data.append(flat_run)
    return flat_data

In [ ]:
data = []
for file in Path(f"{root_path}/data").iterdir():
    if m := name_re.search(file.name):
        data += flatten(json.load(open(file)), {"tasks": int(m.group(1))})
df = pd.DataFrame(data)
df = df[df.comm != 'None']
def rename_gccl(v):
    if not isinstance(v, str):
        return v
    if v == "NCCL":
        return gccl
    else:
        return v
df = df.applymap(rename_gccl)
df

In [ ]:
piv = df.set_index(["tasks", "operation", "msg_size", "kernels", "comm" ])
piv = piv.sort_index()
piv

In [ ]:
piv.xs(("AllToAll", 5), level=("operation", "kernels")).query(f"comm in ['MPI', '{gccl}']")

In [ ]:
piv.query(f"comm in ['MPI', '{gccl}']").xs(5, level="kernels").unstack(["operation", "comm"]).real_time.columns

In [ ]:
piv.query(f"comm in ['MPI', '{gccl}']").xs(5, level="kernels").unstack(["operation", "comm"]).real_time.columns

In [ ]:
tmp_df = piv.query(f"comm in ['MPI', '{gccl}']").xs(5, level="kernels").unstack(["operation", "comm"]).real_time
fig, axs = plt.subplots(3, 2,
                        figsize=(14 * 0.75, 12 * 0.75), dpi=300, sharex=True, sharey=True)
for (operation, comm), ax in zip(tmp_df.columns, axs.flatten()):
    _df = tmp_df[operation][comm].dropna().unstack("msg_size")
    ax.set_prop_cycle(default_cycler)
    ax.set_title(f"{operation} with {comm}")
    _df.plot(ax=ax, legend=True, logx=True, logy=True)
    ax.set_ylabel("Runtime in s")
    ax.set_xlabel("Num GPUs")
fig.savefig(f"{root_path}/img/runtime.png")

In [ ]:
tmp_df = piv.xs(("AllToAll", 5), level=("operation", "kernels")).unstack("comm").real_time
comms = [n for n in tmp_df.columns if n != "MPI"]
fig, axs = plt.subplots(len(comms), 1, figsize=(14 * 0.7, 12 * 0.7), dpi=300, sharex=True)
for comm, ax in zip(comms, axs.flatten()):
    operation = "AllToAll"
    _mpi = tmp_df["MPI"].dropna()
    _nccl = tmp_df[comm].dropna()
    speedup = _mpi / _nccl
    speedup = speedup.unstack("msg_size")
    ax.set_prop_cycle(default_cycler)
    ax.set_title(f"{operation} Speedup {comm} over MPI")
    speedup.plot(ax=ax, legend=True, logx=True, logy=True)
    ax.axhline(1, c="gray", ls="--")
    ax.set_ylabel("Speedup")
    ax.set_xlabel("Num GPUs")
fig.savefig(f"{root_path}/img/all-to-all.png")

In [ ]:
tmp_df = piv.real_time.unstack(["operation", "comm"]).xs(5, level="kernels")
fig, axs = plt.subplots(len(tmp_df.columns.levels[0]), 1, figsize=(14 * 0.7, 12 * 0.7), dpi=300, sharex=True, sharey=True)
for operation, ax in zip(tmp_df.columns.levels[0], axs.flatten()):
    _mpi = tmp_df[operation]["MPI"].dropna()
    _nccl = tmp_df[operation][gccl].dropna()
    speedup = _mpi / _nccl
    speedup = speedup.unstack("msg_size")
    ax.set_prop_cycle(default_cycler)
    ax.set_title(f"{operation} Speedup {gccl} over MPI")
    speedup.plot(ax=ax, legend=True, logx=True, logy=True)
    ax.axhline(1, c="gray", ls="--")
    ax.set_ylabel("Speedup")
    ax.set_xlabel("Num GPUs")
fig.savefig(f"{root_path}/img/speedup.png")

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=160, figsize=(4, 3), layout="tight")
_df = piv.real_time.xs(("AllReduce", 5), level=("operation", "kernels")).unstack(["comm", "msg_size"])
speedup = _df.MPI / _df[gccl]
speedup = speedup.rename(columns=dict((i, i * 8) for i in speedup.columns))
speedup = speedup.rename_axis("Message Size", axis=1)
speedup.plot(ax=ax, logx=True, logy=False)
ax.axhline(1, c="gray", ls="--")
ax.set_title(f"All-Reduce Speedup of {gccl} Over MPI")
ax.set_xlabel("Num GPUs")
ax.set_ylabel("Speedup")
fig.savefig(f"{root_path}/img/all-reduce-speedup.png")

In [ ]:
fig, axs = plt.subplots(1, 2, dpi=160, figsize=(8, 3), sharey=True, layout="tight")
_df = piv.real_time.xs(("AllToAll", 5), level=("operation", "kernels")).unstack(["comm", "msg_size"])
speedup = pd.concat([_df.MPI /_df[gccl], _df.MPI / _df.MPI_NeighborHood], axis=1, keys=[gccl, "Neighborhood"])
for operation, ax in zip(speedup.columns.levels[0], axs.flatten()):
    sp = speedup[operation]
    sp = sp.rename(columns=dict((i, i * 8) for i in sp.columns))
    sp = sp.rename_axis("Message Size", axis=1)
    ax.set_prop_cycle(default_cycler)
    ax.set_title(f"All-To-All Speedup of {operation} over MPI")
    sp.plot(ax=ax, legend=True, logx=True, logy=True)
    ax.axhline(1, c="gray", ls="--")
    ax.set_ylabel("Speedup")
    ax.set_xlabel("Num GPUs")
fig.savefig(f"{root_path}/img/all-to-all-speedup.png", )